In [5]:
pip install -r requirements.txt

  Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement json==2.0.9 (from versions: none)
ERROR: No matching distribution found for json==2.0.9


In [6]:
import pandas as pd
import torch
import requests
import json
from transformers import BertModel, BertTokenizer

In [7]:
tasks = pd.read_excel('./test/tasks.xlsx')
solutions = pd.read_excel('./test/solutions.xlsx')
tests = pd.read_excel('./test/tests.xlsx')

In [8]:
tests_grouped = tests.groupby(['task_id'], dropna=False).agg({
    'type': list,
    'input': list,
    'output': list
}).reset_index()

tasks = tasks.rename(columns={'id': 'task_id'})
merged_df = solutions.merge(tasks[['task_id', 'description', 'author_solution']], on='task_id', how='inner')
merged_df = merged_df.merge(tests_grouped[['task_id', 'type', 'input', 'output']], on='task_id', how='inner')

merged_df['id'] = merged_df['description']


merged_df.drop(columns=['id'], inplace=True)

merged_df['income'] = (
    "Укажи где ошибка в решении: " + merged_df['student_solution'] + " Текст задачи: " + merged_df['description']
    + "тип тестов: " + merged_df['type'].apply(lambda x: ', '.join(x)) + "ввод:" + merged_df['input'].apply(lambda x: ', '.join(map(str, x)))
    + "вывод:" + merged_df['output'].apply(lambda x: ', '.join(map(str, x)))+ '"}]'

)

merged_df = merged_df.rename(columns={'author_comment': 'outcome'})
df = merged_df.loc[:, ['income']]

In [9]:
def make_request(user_text):

    req = {
        "modelUri": "ds://bt10ib0qsr3s3kb5d5rv",
        "completionOptions": {
            "stream": False,
            "temperature": 0.1,
            "maxTokens": "2000"
        },
        "messages": [
            {
                "role": "system",
                "text": "Ты — ИИ-помощник, который помогает студентам в обучении программированию, предоставляя только подсказки и направляющие вопросы. Никогда не давай готовый код или решения, вместо этого побуждай студентов анализировать свои идеи и подходы, чтобы они могли самостоятельно найти ответ."
            },
            {
                "role": "user",
                "text": user_text
            }
        ]
    }

    headers = {
        "Authorization": "Bearer " + 't1.9euelZrGmpCWkI7MjIyLxpaJkpiKze3rnpWanpKRjo2Kmc6Snp2cip3Jnpjl9PdTYTZH-e8NQVWw3fT3ExA0R_nvDUFVsM3n9euelZqNlo3IkJyMyZWWyJbKyJyNj-_8zef1656Vmo7ImMjJjZfLx4uelo2bkIqW7_3F656Vmo2WjciQnIzJlZbIlsrInI2P.-7Dxzw7MyHkH1Mn5ijHELSPunE-93jchLwUV4rVIQ0TGFylqs8uWDY_LBk7JCXsvgrpkanZM1zs44XSN9OJfBg',
        "x-folder-id": "b1g5sk05o8qct0emnmfq"
    }


    res = requests.post("https://llm.api.cloud.yandex.net/foundationModels/v1/completion",
                        headers=headers, json=req)
    print(res.json())
    return res.json()

responces = []
for index, row in df.iterrows():
    user_text = row['income']

    response = make_request(user_text)['result']['alternatives'][0]['message']['text']
    responces.append(response)
    print(f"{index}: {response}")


{'result': {'alternatives': [{'message': {'role': 'assistant', 'text': 'Обратите внимание на некорректные табуляции в сравнении данных.'}, 'status': 'ALTERNATIVE_STATUS_FINAL'}], 'usage': {'inputTextTokens': '374', 'completionTokens': '12', 'totalTokens': '386'}, 'modelVersion': '07.03.2024'}}
0: Обратите внимание на некорректные табуляции в сравнении данных.
{'result': {'alternatives': [{'message': {'role': 'assistant', 'text': 'Обратите внимание на некорректные табуляции в сравнении данных.'}, 'status': 'ALTERNATIVE_STATUS_FINAL'}], 'usage': {'inputTextTokens': '379', 'completionTokens': '12', 'totalTokens': '391'}, 'modelVersion': '07.03.2024'}}
1: Обратите внимание на некорректные табуляции в сравнении данных.
{'result': {'alternatives': [{'message': {'role': 'assistant', 'text': 'Обратите внимание на некорректные табуляции в табу-отступах.'}, 'status': 'ALTERNATIVE_STATUS_FINAL'}], 'usage': {'inputTextTokens': '377', 'completionTokens': '15', 'totalTokens': '392'}, 'modelVersion':

KeyError: 'result'

In [10]:
for index, row in df.iterrows():
  if index>=len(responces):
    user_text = row['income']

    response = make_request(user_text)['result']['alternatives'][0]['message']['text']
    responces.append(response)
    print(f"{index}: {response}")

{'result': {'alternatives': [{'message': {'role': 'assistant', 'text': 'Некорректный синтаксис при использовании функции open.'}, 'status': 'ALTERNATIVE_STATUS_FINAL'}], 'usage': {'inputTextTokens': '826', 'completionTokens': '11', 'totalTokens': '837'}, 'modelVersion': '07.03.2024'}}
136: Некорректный синтаксис при использовании функции open.
{'result': {'alternatives': [{'message': {'role': 'assistant', 'text': 'Некорректный синтаксис при использовании функции open.'}, 'status': 'ALTERNATIVE_STATUS_FINAL'}], 'usage': {'inputTextTokens': '826', 'completionTokens': '11', 'totalTokens': '837'}, 'modelVersion': '07.03.2024'}}
137: Некорректный синтаксис при использовании функции open.
{'result': {'alternatives': [{'message': {'role': 'assistant', 'text': 'Некорректный синтаксис при использовании функции open.'}, 'status': 'ALTERNATIVE_STATUS_FINAL'}], 'usage': {'inputTextTokens': '827', 'completionTokens': '11', 'totalTokens': '838'}, 'modelVersion': '07.03.2024'}}
138: Некорректный синт

KeyError: 'result'

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')
bert_model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased-sentence')
bert_model.eval()  # Установите модель в режим оценки

def get_bert_embeddings(texts):
    inputs = bert_tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = bert_model(**inputs)
    # Получите эмбеддинги из последнего скрытого слоя
    embedding = outputs.last_hidden_state[:, 0, :].squeeze()  # Среднее по всем токенам
    return embedding


In [ ]:
solutions['author_comment'] = responces
solutions['author_comment_embedding']  = [' '.join(map(str, row.tolist())) for row in get_bert_embeddings(responces)]
result = solutions.rename(columns={'id': 'solution_id'}).drop(["task_id", "student_solution"], axis = 1)
result

In [ ]:
result.to_csv("results.csv", index = False)

